# Articuno Schema Inference Demo

In [ ]:
# Install Articuno (if needed)
# !pip install articuno[polars,pandas]

In [1]:
import articuno as art
from pydantic import BaseModel

## Example 1: Infer Model from Iterable of Dictionaries

In [2]:

records = [
    {"id": 1, "name": "Alice", "age": 30},
    {"id": 2, "name": "Bob", "age": None},
    {"id": 3, "name": "Charlie", "age": 25},
]

UserModel = art.infer_pydantic_model(records, model_name="UserModel")
print(UserModel.model_json_schema())


{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'title': 'Name', 'type': 'string'}, 'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Age'}}, 'required': ['id', 'name'], 'title': 'UserModel', 'type': 'object'}


In [3]:

instances = art.df_to_pydantic(records, model=UserModel)

for instance in instances:
    print(instance)


id=1 name='Alice' age=30
id=2 name='Bob' age=None
id=3 name='Charlie' age=25


In [4]:
print(art.generate_class_code(UserModel))

# generated by datamodel-codegen:
#   filename:  schema.json
#   timestamp: 2025-07-08T19:47:51+00:00

from __future__ import annotations

from typing import Optional

from pydantic import BaseModel, Field


class UserModel(BaseModel):
    id: int = Field(..., title='Id')
    name: str = Field(..., title='Name')
    age: Optional[int] = Field(None, title='Age')



## Example 2: Using a Generator as Input

In [5]:

def record_generator():
    yield {"id": 101, "score": 95.5}
    yield {"id": 102, "score": 88.0}
    yield {"id": 103, "score": 92.0}

ScoreModel = art.infer_pydantic_model(record_generator(), model_name="ScoreModel")
print(ScoreModel.model_json_schema())


{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'score': {'title': 'Score', 'type': 'number'}}, 'required': ['id', 'score'], 'title': 'ScoreModel', 'type': 'object'}


In [6]:

score_instances = art.dicts_to_pydantic(record_generator(), model=ScoreModel)

for score in score_instances:
    print(score)


id=101 score=95.5
id=102 score=88.0
id=103 score=92.0


## Example 3: Automatically Inferring and Creating Instances

In [7]:

json_data = [
    {"product": "Apple", "price": 0.99, "quantity": 10},
    {"product": "Banana", "price": 0.59, "quantity": 20},
    {"product": "Cherry", "price": 3.99, "quantity": 5},
]

product_instances = art.dicts_to_pydantic(json_data)

for product in product_instances:
    print(product)


quantity=10 product='Apple' price=0.99
quantity=20 product='Banana' price=0.59
quantity=5 product='Cherry' price=3.99
